In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import time
import nltk
from nltk.corpus import stopwords
# from nltk.stem import French
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
from spacy.lang.en.stop_words import STOP_WORDS as en_stop
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer # package for french lemmatization 
import random
import re

seed = 10
random.seed(seed)
np.random.seed(seed)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\samue\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\samue\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\samue\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\samue\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
# to find P(Y=k), just do test_labels.where(k)/len(test_labels)
# vectos_train will contain all the information I need to create the counting for each word
# use the get_feature_names to create a dictionary to map to all counts
# this dictionary would be contained in another dict that has the labels as keys
# to find P(xj,k), find the indices where test_labels = (k), then find

In [ ]:
# TODO: Handle cases where two examples are permutations of another
# TODO: Handle laplace smoothing, i.e. word from corpus not in word list

In [ ]:
# worth looking into stripping accents, and sklearn feature extraction libarary including idf extractor

In [13]:
# path = '/content/drive/MyDrive/Olympus in the Sky/McGill/2024 - Winter/ECSE 551/Data/'
path = 'Data/' # sam path VScode
df = pd.read_csv(path + 'train.csv', encoding = "ISO-8859-1")

training_data = df['body'].apply(lambda x: x.replace('_', ' ')).to_numpy()
training_labels = df['subreddit'].to_numpy()

indices = np.random.permutation(len(training_data))
training_data = training_data[indices]
training_labels = training_labels[indices]

test_split = 0.2

(training_data, testing_data, training_labels, testing_labels) = train_test_split(training_data, training_labels,  test_size = int(len(training_data)*test_split), random_state=seed)

spacy_stopwords_list = list(fr_stop) + list(en_stop)
nltk_stopwords_list = stopwords.words('english') + stopwords.words('french')

In [6]:
class LemmaTokenizer:
     def __init__(self):
       self.wnl = WordNetLemmatizer()
     def __call__(self, doc):
       return [self.wnl.lemmatize(t,pos ="v") for t in word_tokenize(doc) if t.isalpha()]

In [7]:
class NB():
  def __init__(self):
    spacy_stopwords_list = list(fr_stop) + list(en_stop)
    nltk_stopwords_list = stopwords.words('english') + stopwords.words('french')
    # self.vectorizer = CountVectorizer(binary = True, max_features = 3000, stop_words = list(set().union(spacy_stopwords_list, nltk_stopwords_list)), tokenizer=LemmaTokenizer())
    self.vectorizer = CountVectorizer(binary = True, max_features = 3000, stop_words = list(set().union(spacy_stopwords_list, nltk_stopwords_list)))
    self.no_pred = []

  # currently train_corpus expects a np array containing a list of strings, not array of words, nxm
  # test_label is an nx1 array
  def train(self, train_corpus, test_labels):
    assert len(train_corpus)  == len(test_labels)
    assert type(train_corpus) == type(test_labels) == np.ndarray

    vectors_train    = self.vectorizer.fit_transform(train_corpus).todense()
    self.word_list   = self.vectorizer.get_feature_names_out()
    self.num_samples = len(test_labels)
    self.word_count  = dict(zip(self.word_list, np.array(vectors_train.sum(axis=0))[0]))

    unique, counts   = np.unique(test_labels, return_counts=True)
    self.label_count = dict(zip(unique, counts))
    self.labels      = unique

    self.word_count_given_label = {}
    for label in self.labels:
      indices = np.where(test_labels == label)[0]
      tot_word_count = np.array(vectors_train[indices].sum(axis=0))[0]
      self.word_count_given_label[label] = {self.word_list[i] : tot_word_count[i] for i in range(len(tot_word_count))}
      # print(label, self.word_count_given_label[label])

  # Assuming test_corpus is 2-d array where each test sample is a string
  def predict(self, test_corpus):

    predictions = []
    pattern = re.compile(r"(?u)\b\w\w+\b")
    for index, corpus in enumerate(test_corpus):

      best_label = ''
      best_prob  = -np.inf

      # Text processing based on CountVectorizer's regex
      # corpus = corpus.lower().replace('_', ' ')
      # corpus_words = list(set(re.findall(pattern, corpus)))

      corpus = corpus.replace('_', ' ')
      try:
        _ = self.vectorizer.fit_transform([corpus])
      except:
        self.no_pred.append(corpus)
        best_label = random.choice(self.labels)
        predictions.append(best_label)
        continue
      corpus_words  = self.vectorizer.get_feature_names_out()

      for label in self.labels:
        p_of_y = self.label_count[label]/self.num_samples

        p_of_x_given_y = 1

        for word in self.word_list:
          if word in corpus_words:
            xj = 1
          else:
            xj = 0

          theta_xj_k      = (self.word_count_given_label[label][word] + 1) / (self.label_count[label] + len(self.labels))
          p_of_x_given_y *= (theta_xj_k**(xj) * (1-theta_xj_k)**(1-xj))
          # print(label, word, p_of_x_given_y, theta_xj_k, (theta_xj_k**(xj) * (1-theta_xj_k)**(1-xj)))

        unseen_words = [new_word for new_word in corpus_words if new_word not in self.word_list]
        for word in unseen_words:
          # p_of_x_given_y *= 1/(self.label_count[label] + len(self.labels))
          p_of_x_given_y *= 1/(len(self.labels)) # When there are a large number of unseen words, the p_of_x_given_y basically becomes 0
        # print(len(unseen_words), len(corpus_words))

        p_of_y_given_x = np.log(p_of_y * p_of_x_given_y)
        # print(p_of_y, p_of_x_given_y)
        # print(label, p_of_y_given_x)

        # print(p_of_y_given_x, best_prob)
        if p_of_y_given_x > best_prob:
          best_prob = p_of_y_given_x
          best_label = label

      if best_label == '':
        self.no_pred.append(corpus)
      #   best_label = random.choice(self.labels)

      predictions.append(best_label)

    return predictions

In [54]:

# from nltk.stem import WordNetLemmatizer
# from nltk.stem import FrenchLefffLemmatizer
# from nltk.tokenize import word_tokenize
# import nltk
# nltk.download('punkt')
from nltk.tokenize.treebank import TreebankWordDetokenizer
spacy_stopwords_list = list(fr_stop) + list(en_stop)
nltk_stopwords_list = stopwords.words('english') + stopwords.words('french')

stop_words = list(set().union(spacy_stopwords_list, nltk_stopwords_list))

def lemmatizer(sentence):
    # Tokenize the sentence
    new_sentence_list = []
    for sentence in sentences:
        tokens = word_tokenize(sentence)
        
        words_w_stopwords = [i for i in tokens if i not in stop_words]

        is_english = all(ord(char) < 128 for char in sentence)
        if is_english:
            lemmatizer = WordNetLemmatizer()
        else:
            lemmatizer = FrenchLefffLemmatizer()

        # Lemmatize the tokens
        lemmatized_tokens = [lemmatizer.lemmatize(word) for word in words_w_stopwords]

        new_words = TreebankWordDetokenizer().detokenize(lemmatized_tokens)
        # print(new_words)
        new_sentence_list.append(new_words)
    
    return new_sentence_list
    

# Example mixed set of sentences (some English, some French)
sentences = [
    "My car is beautiful.",             # English
    "€a arrive, tous les rassemblements du genre o— les gens sont tass‚s les uns … c“t‚ des autres, sont un paradis des pickpockets. gloofest est sp‚cial parce-que le monde portent un gros manteau d'hiver ‚pais et … cause de la musique forte et des gens qui se bousculent en dansant, ce qui rend plus facile le vol sans que les victimes ne s'en rendent compte.",       # French
    "He plays football.",              # English
    "Elle joue au football.",          # French
    "They are students.",              # English
    "Ils sont étudiants."              # French
]

# Tokenize and lemmatize each sentence in the mixed set

    # print(lemmatizer(sentence))
print(lemmatizer(sentences))

['My beautiful.', "€a arrive, rassemblement genre o— tass‚s … “ t‚, paradis pickpocket . gloofest sp‚cial parce-que monde portent gros manteau d'hiver ‚pais … cause musique forte bousculent dansant, facile vol victime s'en rendent compte.", 'He play football.', 'Elle joue football.', 'They student.', 'Ils étudiant.']


In [8]:
def k_fold_validation(x_train_data, y_train_data, K = 10):
  assert type(x_train_data) == np.ndarray and type(y_train_data) == np.ndarray

  len_of_data  = len(x_train_data) - len(x_train_data) % K
  data_x       = x_train_data[:len_of_data]
  data_y       = y_train_data[:len_of_data]
  size_of_fold = len(x_train_data[:len_of_data]) // K
  validation_error = 0

  error    = []
  pred     = []
  timings  = []
  no_preds = []

  naive_bayes = NB()

  for i in range(K):

    naive_bayes = NB()

    if i != K-1:
      validation_fold_x = data_x[i*size_of_fold:(i+1)*size_of_fold]
      validation_fold_y = data_y[i*size_of_fold:(i+1)*size_of_fold]

      training_folds_x  = np.concatenate((data_x[:i*size_of_fold], data_x[(i+1)*size_of_fold:]))
      training_folds_y  = np.concatenate((data_y[:i*size_of_fold], data_y[(i+1)*size_of_fold:]))

    else:
      validation_fold_x = data_x[i*size_of_fold:]
      validation_fold_y = data_y[i*size_of_fold:]

      training_folds_x  = data_x[:i*size_of_fold]
      training_folds_y  = data_y[:i*size_of_fold]

    start_time = time.time()
    naive_bayes.train(training_folds_x, training_folds_y)
    end_time = time.time()

    timings.append(end_time - start_time)

    pred_valid = naive_bayes.predict(validation_fold_x)
    no_pred = naive_bayes.no_pred
    pred_train = naive_bayes.predict(training_folds_x)

    fold_error = {}
    fold_error['validation'] = 1 - accuracy_score(pred_valid, validation_fold_y)
    validation_error        += 1 - accuracy_score(pred_valid, validation_fold_y)
    fold_error['train']      = 1 - accuracy_score(pred_train, training_folds_y)

    model_pred = {}
    model_pred['validation'] = (pred_valid, validation_fold_y)
    model_pred['train']      = (pred_train, training_folds_y )

    error.append(fold_error)
    pred.append(model_pred)
    no_preds.append(no_pred)

  info = {'error': error, 'pred': pred, 'time': timings, 'no_preds' : no_preds}

  return validation_error/K, info

# Simple test cases for NB

In [9]:
train_corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
     'Is this the first document?',]
test_labels = np.array(['1', '2', '3', '4'])
train_corpus = np.array(train_corpus)
test_corpus = ['harro first pink haha']
model = NB()
model.train(train_corpus, test_labels)
model.predict(test_corpus)

c:\Users\samue\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neuf', 'quelqu'] not in stop_words.
  warnings.warn(


['3']

In [8]:
# Training data
X_train = np.array([
    "I love this movie",
    "This movie is great",
    "A movie like this is great",
    "I hate this movie",
    "This movie is terrible"
])

# Corresponding labels
y_train = np.array([1, 1, 1, 0, 0])  # 1 for positive sentiment, 0 for negative sentiment


model = NB()
model.train(X_train, y_train)
X_test = [
    "I love this movie terrible", # Example of both class being viable but the second being chosen due to label eval order in NB
    "I hate this great movie",
    "This movie is terrible"
]


print(model.predict(X_test))

[0, 1, 0]


# K-fold for NB

In [ ]:
# TODO: How to select best performing split
# TODO: No predictions given, divide by zero warnings might be the cause (Solved?)
# TODO: Prune weird words from word lsit like numebrs and etc
# TODO: Add lemitization
# TODO: Test changing the prob caluclation to be a sum of logs
# TODO: Lemmitization for french and english

In [9]:
model = NB()
model.train(training_data, training_labels)
pred = model.predict(testing_data)
(pred == testing_labels).sum()/len(testing_labels)

0.6357142857142857

In [10]:
data = model.no_pred

debug = NB()
debug.train(training_data, training_labels)
pred = debug.predict(data)
pred

[]

In [11]:
mean_error, info = k_fold_validation(training_data, training_labels)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neuf', 'quelqu'] not in stop_words.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neuf', 'quelqu'] not in stop_words.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neuf', 'quelqu'] not in stop_words.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neuf', 'quelqu'] not in stop_words.
  warning

In [12]:
mean_error

0.3589285714285714

In [13]:
info['error']

[{'validation': 0.3392857142857143, 'train': 0.08730158730158732},
 {'validation': 0.3035714285714286, 'train': 0.08730158730158732},
 {'validation': 0.4196428571428571, 'train': 0.10317460317460314},
 {'validation': 0.4017857142857143, 'train': 0.0982142857142857},
 {'validation': 0.3303571428571429, 'train': 0.08333333333333337},
 {'validation': 0.3660714285714286, 'train': 0.09226190476190477},
 {'validation': 0.4464285714285714, 'train': 0.09523809523809523},
 {'validation': 0.3482142857142857, 'train': 0.10515873015873012},
 {'validation': 0.3214285714285714, 'train': 0.09126984126984128},
 {'validation': 0.3125, 'train': 0.08234126984126988}]

In [14]:
info['no_preds']

[['?????????????????????? ?????????????????????? ?????????????????????? ?????????????????????? ?????????????????????? ?????????????????????? ?????????????????????? ?????????????????????? ?????????????????????? ?????????????????????? ?????????????????????? ?????????????????????? ?????????????????????? ?????????????????????? ?????????????????????? ?????????????????????? ?????????????????????? ?????????????????????? ?????????????????????? ?????????????????????? ?????????????????????'],
 ['?????????????????????? ?????????????????????? ?????????????????????? ?????????????????????? ?????????????????????? ?????????????????????? ?????????????????????? ?????????????????????? ?????????????????????? ?????????????????????? ?????????????????????? ?????????????????????? ?????????????????????? ?????????????????????? ?????????????????????? ?????????????????????? ?????????????????????? ?????????????????????? ?????????????????????? ?????????????????????? ?????????????????????'],
 ['?????????????????????

In [17]:
model.word_list

array(['00', '000', '02', ..., 'zero', 'zone', 'ÿÿ'], dtype=object)

In [ ]:
sorted(model.word_count.items(), key=lambda item: item[1], reverse = True)[:-1]

# Actual Test Case

In [19]:
model = NB()
model.train(training_data, training_labels)

test_df = pd.read_csv(path + 'test.csv', encoding = "ISO-8859-1")
test_data = test_df['body'].to_numpy()

predictions = model.predict(test_data)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neuf', 'quelqu'] not in stop_words.
  warnings.warn(


In [115]:
pd.DataFrame({'Subreddit':predictions}).reset_index().rename(columns={"index": "Id"}).to_csv('results.csv', index=False)

# Extra

In [ ]:
%pip install wordninja
import wordninja

In [ ]:
' '.join(wordninja.split('This document is the spider-man document,egxnd3mtd2l6lxnlcnaih0j1cybhbmqgq29hy2ggq2hhcnrlcibtzwxib3vybmuybhaagbyyhjileaaygaqyiguyhgmycxaagiaegiofgiydmgsqabiabbikbrigazileaaygaqyiguyhgmycxaagiaegiofgiydsoobuksdwkkbcaf4azabajgbnwkgaewnqgedmi04uaedyaea.'))

'This document is the spider man document eg x nd 3 m td 2 l 6 l xn lc nai h 0 j 1 cy bh bm qg q 29 hy 2 g gq 2 h hcn rlc ibt zw xi b 3 vy b muy b haag by y hj ilea a yg aq yi guy hg my cx a agia eg i of gi yd mg sq abi abb ik brig a zile a a yg aq yi guy hg my cx a agia eg i of gi yd soo buk s dw k kb caf 4 az abaj gb nw kg a ew n qg edm i 04 uae dy a ea'

In [21]:
sorted(model.word_count.items(), key=lambda item: item[1], reverse = True)[-100:]

[('target', 3),
 ('tarif', 3),
 ('tarifs', 3),
 ('tea', 3),
 ('tech', 3),
 ('technically', 3),
 ('tends', 3),
 ('terminal', 3),
 ('thermique', 3),
 ('thoughts', 3),
 ('thousand', 3),
 ('thrown', 3),
 ('til', 3),
 ('toast', 3),
 ('toilets', 3),
 ('tol', 3),
 ('tool', 3),
 ('tops', 3),
 ('totally', 3),
 ('tottenham', 3),
 ('tough', 3),
 ('towns', 3),
 ('trajet', 3),
 ('trans', 3),
 ('trash', 3),
 ('travailleurs', 3),
 ('travelled', 3),
 ('trees', 3),
 ('triomphe', 3),
 ('trottinette', 3),
 ('trouvent', 3),
 ('trucks', 3),
 ('trump', 3),
 ('tudiants', 3),
 ('tunnel', 3),
 ('turning', 3),
 ('typical', 3),
 ('ues', 3),
 ('ukraine', 3),
 ('understood', 3),
 ('unemployed', 3),
 ('units', 3),
 ('universit', 3),
 ('unlikely', 3),
 ('unpopular', 3),
 ('unsure', 3),
 ('update', 3),
 ('ups', 3),
 ('urbain', 3),
 ('urban', 3),
 ('urgence', 3),
 ('urgent', 3),
 ('users', 3),
 ('ussi', 3),
 ('valeur', 3),
 ('valuable', 3),
 ('vast', 3),
 ('venue', 3),
 ('verbal', 3),
 ('viable', 3),
 ('victimes', 3),